# Assignment 2

The objective of this assignment was to create and run a market making strategy on the data that has been retrieved in assigment 1. This strategy consists on placing bids and asks on the market at different refresh times and idealy capture the spread. The ideal market condition for this strategy would be a range bound market where the market is not trending in any direction. In such markets, the probability of both the orders getting executed is very high. In trending markets, however, it's important to have a loss reduction mechanism. The following strategy is created using Pandas.

# Assumptions

1. The Traded quantity of historical data can be determined by a restraint on either BTC or USDT. For example, an order can be open to buy BTC with USDT but there might not be another order to buy USDT with BTC at the same point in time at the same price and vice versa. Liquidity could have been available on one side of the market but it's not possible to find out which side without looking at the order cook for that point in time. So, for this strategy, it is assumed that if the price crosses an orde, it is executed with the defined value.

2. The notional limit on one trade is 1000USD and the maximum notional value on one side is 5000USD. So the notional value of every trade taken is equal to 1000USD and a maximum of 5 trades are open at any point in time before hedging or exiting the position.

3. No transaction costs or slippage

4. It is assumed that if the traded price crosses the bid or the ask, our order will always be the best order in the market and hence the trade happens whenever it is triggered. 

Step 1: Import the data from the previous assignment.

In [2]:
import backtrader as bt
import pandas as pd
from datetime import datetime
pd.set_option('display.max_rows', 900)

In [3]:
%store -r data

Step 2: Create a tradesheet to record the trades 

In [4]:
tradesheet = pd.DataFrame(columns = ['type','spot','buy','buy_timeStamp','sell','sell_timeStamp','pnl'])

Step 3: Specify all the variables that will be used

In [5]:
bid_margin = 0.0003 
ask_margin = 0.0003
balance = 1000 #The nominal value(USDT) for each trade, this will be used to convert to BTC for actual trade quantity
buy_position = False # if the position(buy or sell) is false, it means that the position is open and not executed yet
sell_position = False
opening_time = data.loc[0,'timeStamp']
opening_spot = data.loc[0,'tradedPrice'] # opening spot price of BTCUSDT
buy_order = opening_spot * (1-bid_margin) 
sell_order = opening_spot * (1+ask_margin)
trade_id = 0 #counts the number of trades and also matches a buy order to a sell order
refresh_timeStamp = opening_time
carry_buy = {} #A dictionary that holds all the open buy trades that did not execute within the refresh time, so they will be carried forward to the next refresh rate
carry_sell = {}#A dictionary that holds all the open sell trades that did not execute within the refresh time, so they will be carried forward to the next refresh rate
carry_buy[buy_order] = trade_id #matching a buy order to it's trade id
carry_sell[sell_order] = trade_id #matching a sell order to it's trade id
refresh_time = 360000 # quote refresh time in milliseconds(6 minutes)

Step 4: Create the main algorithm that will loop through the data and perform the mentioned strategy.

In [6]:
for i in data.index:
    #Since there will a latency of 100ms between the algorithm plaacing the order and the order reaching the exchange, any order matches that happen in between this time wont be considered as executable trades
    if (data.loc[i,'timeStamp'] + 100 > refresh_timeStamp) and (data.loc[i,'timeStamp'] < refresh_timeStamp + refresh_time):
        if (buy_position == False) and (sell_position == False):
            if data.loc[i,'tradedPrice'] > sell_order and (len(carry_sell) < 6):
                sell = (balance/data.loc[i,'tradedPrice']) #converting USDT to BTC
                sell_position = True
                tradesheet.loc[trade_id,'spot'] = data.loc[i,'tradedPrice']
                tradesheet.loc[trade_id,'sell'] = sell
                tradesheet.loc[trade_id,'sell_timeStamp'] = data.loc[i,'timeStamp']
                tradesheet.loc[trade_id,'type'] = 1
                continue
            elif data.loc[i,'tradedPrice'] < buy_order and (len(carry_buy) < 6):
                buy = (balance/data.loc[i,'tradedPrice'])  #converting USDT to BTC
                buy_position = True
                tradesheet.loc[trade_id,'spot'] = data.loc[i,'tradedPrice']
                tradesheet.loc[trade_id,'buy'] = buy
                tradesheet.loc[trade_id,'buy_timeStamp'] = data.loc[i,'timeStamp']
                tradesheet.loc[trade_id,'type'] = 0

                continue
        elif (buy_position == False) and (sell_position == True) and (len(carry_sell) < 6):
            if data.loc[i,'tradedPrice'] < buy_order:
                buy = (balance/data.loc[i,'tradedPrice'])
                buy_position = True
                tradesheet.loc[trade_id,'buy'] = buy
                tradesheet.loc[trade_id,'buy_timeStamp'] = data.loc[i,'timeStamp']

                continue
        elif (buy_position == True) and (sell_position == False) and (len(carry_buy) < 6):
            if data.loc[i,'tradedPrice'] > sell_order:
                sell = (balance/data.loc[i,'tradedPrice'])
                sell_position = True
                tradesheet.loc[trade_id,'sell'] = sell
                tradesheet.loc[trade_id,'sell_timeStamp'] = data.loc[i,'timeStamp']
                continue
        elif(buy_position == True) and (sell_position == True):
            #if both trades are executed don't create more trades until the next refresh time
            pass
        
        #checking for old trades
        for values in list(carry_buy.keys()):
            if data.loc[i,'tradedPrice'] < values:
                tradesheet.loc[carry_buy[values],'buy'] = (balance/data.loc[i,'tradedPrice'])
                tradesheet.loc[carry_buy[values],'buy_timeStamp'] =data.loc[i,'timeStamp']
                carry_buy.pop(values)
            else:
                continue
        for values in list(carry_sell.keys()):
            if data.loc[i,'tradedPrice'] > values:
                tradesheet.loc[carry_sell[values],'sell'] = (balance/data.loc[i,'tradedPrice'])
                tradesheet.loc[carry_sell[values],'sell_timeStamp'] = data.loc[i,'timeStamp']
                carry_sell.pop(values)
            else:
                continue
    elif data.loc[i,'timeStamp'] > refresh_timeStamp + refresh_time:
        refresh_timeStamp = data.loc[i,'timeStamp'] 

        # if both the trades are not executed, cancel both trades
        if (buy_position == False) and (sell_position == False):
            try:
                carry_buy.pop(buy_order)
                carry_sell.pop(sell_order)
            except Exception as e:
                print(e)
        trade_id += 1
        buy_position = False
        sell_position = False
        refresh_spot = data.loc[i,'tradedPrice']
        buy_order = refresh_spot * (1-bid_margin)
        carry_buy[buy_order] = trade_id
        sell_order = refresh_spot * (1+ask_margin)
        carry_sell[sell_order] = trade_id
        if len(carry_sell) > 5:
            exit_id = sorted(carry_sell.values())[0]
            tradesheet.loc[exit_id,'sell'] = (balance/refresh_spot)
            tradesheet.loc[exit_id,'sell_timeStamp'] = refresh_timeStamp
            for key,value in carry_sell.items():
                if value == exit_id:
                    carry_sell.pop(key)
                    break
        if len(carry_buy) > 5:
            exit_id = sorted(carry_buy.values())[0]
            tradesheet.loc[exit_id,'buy'] = (balance/refresh_spot)
            tradesheet.loc[exit_id,'buy_timeStamp'] = refresh_timeStamp
            for key,value in carry_buy.items():
                if value == exit_id:
                    carry_buy.pop(key)
                    break
            

Explanation:

The algorithm first defines parameters that decide the first trade( first buy order, first sell order, first time stamp and refresh time). Then, the latency is taken into consideration(skips the data which falls before the latency time)  and the algorithm goes through all the tick data for a buy order or a sell order match that can be executed before the next refresh time. At this point, there are four possibilities:

Possibility 1: Both buy and sell orders are executed within the refresh time. In this case, no new trades are taken before until the next refresh time.
Possibility 2: Only buy order is executed and sell order is open. In this case, the open sell orders are added to a dictionary called 'carry_sell' are carried forward to the next refresh time.
Possibility 3: Only sell order is executed and buy order is open. In this case, the open buy orders are added to a dictionary called 'carry_buy' are carried forward to the next refresh time.
Possibility 4: Both orders are not executed within the refresh time. In this case, both the orders are canceled and new orders are created at the refresh time.
The code for this part of the strategy can be found between line 1 and line 39.

After a particular tick is checked as to whether it has hit both buy and sell orders, the previously carried forward orders are checked and any matches will be executed. The code for this part of the strategy can be found between line 40 and line 55.

It is important to note that only 5 open orders(5000 notional value) are allowed. So if the carry forward orders exceed 5, the oldest one will be executed at curernt price. The algorithm also stops quoting on the side on which the limit is reached.( An alternative to exiting i.e hedging has been discussed below)

When the algorithm reaches the next reset time, it checks for any openn positions and adds them to their respective carry dictionaries. It also resets resets the spot price and creates a new buy order and a sel order. The code for this part of the strategy can be found between line 56 and line 90.



Alternative:

One way to not exit at market price and hedge our loss is to enter a counter position is BTCUSDT perpetual futures. This position can be until the market comes back to the exectution price. If the market does not come back to the execution price, both the spot and perpetual futures postions can be unwinded at a defined loss that is fixed at the moment of entering the fuures hedge. This approach is not involved in the above code due the notional value constaint that's present and taking a new position would double the notional capital required. If the notional value constraint can be ignored, the data can be retrieved using the same code in Assignment 1 with https://data.binance.vision/data/futures/cm/daily/trades/ being the endpoint.

Result:

In [7]:
tradesheet

,type,spot,buy,buy_timeStamp,sell,sell_timeStamp,pnl
0,1,43173.65,0.023177,1646100350736,0.023162,1646092801258,NaN
1,0,43343.62,0.023071,1646093162825,0.023057,1646093314207,NaN
2,1,43573.62,0.022965,1646093528357,0.02295,1646095564980,NaN
3,0,43255.96,0.023118,1646093880472,0.023103,1646093958337,NaN
4,0,43244.84,0.023124,1646094246544,0.023108,1646094253563,NaN
5,0,43364.4,0.023061,1646094603395,0.023046,1646094822079,NaN
6,1,43533.86,0.022986,1646094986610,0.022969,1646094967521,NaN
7,0,43450.84,0.023015,1646095381353,0.023,1646095408567,NaN
8,0,43452.53,0.023015,1646095681009,0.022998,1646096056984,NaN
9,1,43441.79,0.023034,1646097090378,0.023019,1646096043599,NaN


In [8]:
# to export this tradesheet to the next assignment:

%store tradesheet

Stored 'tradesheet' (DataFrame)
